## import file

In [29]:
import pandas as pd
from pathlib import Path

p = Path("/Users/dengshuyue/Desktop/SDOH/analysis/output/cov_addv6_99_23.parquet")
df_my_cov_aligned_short = pd.read_parquet(p)  # uses pyarrow/fastparquet if available
print(df_my_cov_aligned_short.shape)
df_my_cov_aligned_short.head()


(128809, 64)


,SEQN,SDDSRVYR,sdmvpsu,sdmvstra,RIDAGEYR,SEX,RACE,re,household_size,DMDHHSIZ,...,drinking,alcg2,perE_alco,METSCORE_fromPAQ,LTPA_fromPAQ,chol_rx200,SMK_STATUS_STD,CIGS_PER_DAY_CAT,bmi_lu,bmi_cat_new
0,1,1.0,1,5,2.0,F,4.0,Other Hispanic,3,3.0,...,NaN,<NA>,0.000000,NaN,NaN,<NA>,<NA>,<NA>,14.90,UNDER
1,2,1.0,3,1,77.0,M,3.0,Mexican American,1,1.0,...,0.065753,2,0.000000,NaN,NaN,1,NEVER,<NA>,24.90,NORMAL
2,3,1.0,2,7,10.0,F,3.0,Mexican American,4,4.0,...,NaN,<NA>,0.000000,NaN,NaN,0,<NA>,<NA>,17.63,UNDER
3,4,1.0,1,2,1.0,M,4.0,Other Hispanic,7,7.0,...,NaN,<NA>,0.000000,NaN,NaN,<NA>,<NA>,<NA>,15.20,UNDER
4,5,1.0,2,8,49.0,M,3.0,Mexican American,3,3.0,...,1.714286,2,9.101101,NaN,NaN,1,FORMER,<NA>,29.10,OVER


In [30]:
df_my_cov_aligned_short.columns

Index(['SEQN', 'SDDSRVYR', 'sdmvpsu', 'sdmvstra', 'RIDAGEYR', 'SEX', 'RACE',
       're', 'household_size', 'DMDHHSIZ', 'EDU', 'pir', 'met_hr',
       'SMK_STATUS', 'CIGS_PER_DAY', 'FORMER_SMOKER', 'METSCORE', 'LTPA',
       'DIABE', 'HYPERTEN', 'chol_rx', 'CVD', 'cancer', 'probable_depression',
       'ahei_total', 'unemployment2', 'ins', 'HOQ065', 'marriage', 'SNAP',
       'FS', 'WTINT2YR', 'WTMEC2YR', 'WTSAF2YR', 'WTINT4YR', 'WTMEC4YR',
       'WTPH2YR', 'WTINTPRP', 'WTMECPRP', 'WTSAFPRP', 'wt_int', 'wt_mec',
       'wt_fasting', 'wt_phlebotomy', 'marriage_prev', 'marriage_label',
       'marriage3', 'SNAP_src', 'SNAP_bin', 'SNAP_src_rank', 'SNAP_indiv_only',
       'SNAP_indiv_plus_singleton', 'bmi', 'RIAGENDR', 'drinking', 'alcg2',
       'perE_alco', 'METSCORE_fromPAQ', 'LTPA_fromPAQ', 'chol_rx200',
       'SMK_STATUS_STD', 'CIGS_PER_DAY_CAT', 'bmi_lu', 'bmi_cat_new'],
      dtype='object')

## add sleep 05-23 

In [31]:
####

In [32]:
import pandas as pd, numpy as np, io, requests
from typing import Optional

# ----------------- helpers -----------------
def _fetch_xpt(folder: str, filebase: str) -> pd.DataFrame:
    url = f"https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/{folder}/DataFiles/{filebase}.xpt"
    r = requests.get(url, timeout=60); r.raise_for_status()
    df = pd.read_sas(io.BytesIO(r.content), format="xport", encoding="latin1")
    df.columns = [c.upper() for c in df.columns]
    return df

def _safe_fetch(folder: str, filebase: str) -> Optional[pd.DataFrame]:
    try:
        return _fetch_xpt(folder, filebase)
    except Exception as e:
        print(f"[skip] {folder}/{filebase}.xpt — {type(e).__name__}: {e}")
        return None

def _hhmm_to_min(x):
    try:
        s = str(x)
        if ":" not in s: return np.nan
        h, m = s.split(":")
        return int(h) * 60 + int(m)
    except Exception:
        return np.nan

def _build_sleep_vars(slq: pd.DataFrame) -> pd.DataFrame:
    out = slq[["SEQN"]].copy()
    # raw (np.nan when absent so numerics don’t choke)
    out["SLQ050"] = slq["SLQ050"] if "SLQ050" in slq.columns else np.nan
    out["SLD012"] = pd.to_numeric(slq["SLD012"], errors="coerce") if "SLD012" in slq.columns else np.nan
    out["SLD013"] = pd.to_numeric(slq["SLD013"], errors="coerce") if "SLD013" in slq.columns else np.nan
    out["SLD010H"] = pd.to_numeric(slq["SLD010H"], errors="coerce") if "SLD010H" in slq.columns else np.nan
    out["SLQ300"]  = slq["SLQ300"] if "SLQ300" in slq.columns else np.nan
    out["SLQ310"]  = slq["SLQ310"] if "SLQ310" in slq.columns else np.nan

    # weekday hours: SLD012 → SLD010H → SLQ300/SLQ310
    sleep_wk = pd.to_numeric(out["SLD012"], errors="coerce")
    if sleep_wk.isna().all():
        sleep_wk = pd.to_numeric(out["SLD010H"], errors="coerce")
    if sleep_wk.isna().all() and ("SLQ300" in out.columns and "SLQ310" in out.columns):
        t_sleep = out["SLQ300"].apply(_hhmm_to_min)
        t_wake  = out["SLQ310"].apply(_hhmm_to_min)
        dur_min = (t_wake - t_sleep) % (24 * 60)
        sleep_wk = np.round(dur_min / 60.0, 1)

    out["sleep_hours_weekday"] = pd.to_numeric(sleep_wk, errors="coerce")
    out["sleep_hours_weekend"] = pd.to_numeric(out["SLD013"], errors="coerce")
    out["sleep_hours_usual"] = out["sleep_hours_weekday"]
    both = out["sleep_hours_weekday"].notna() & out["sleep_hours_weekend"].notna()
    out.loc[both, "sleep_hours_usual"] = (5*out.loc[both,"sleep_hours_weekday"] + 2*out.loc[both,"sleep_hours_weekend"])/7

    # trouble sleeping (1/0/<NA>)
    ts = pd.to_numeric(out["SLQ050"], errors="coerce")
    out["trouble_sleeping"] = pd.Series(np.where(ts == 1, 1, np.where(ts == 2, 0, np.nan))).astype("Int64")
    return out

# ----------------- cycles (aligned to your SDDSRVYR map) -----------------
targets = [
    ("2005", "SLQ_D", 4),
    ("2007", "SLQ_E", 5),
    ("2009", "SLQ_F", 6),
    ("2011", "SLQ_G", 7),
    ("2013", "SLQ_H", 8),
    ("2015", "SLQ_I", 9),
    ("2017", "SLQ_J", 10),   # 2017–2018
    ("2017", "P_SLQ", 66),   # 2017–Mar 2020 combined (lives under 2017 folder)
    ("2021", "SLQ_L", 12),   # 2021–2023 (no SLQ050 by design)
]

frames = []
for folder, filebase, sdd in targets:
    df = _safe_fetch(folder, filebase)
    if df is None:
        continue
    out = _build_sleep_vars(df)
    out["SDDSRVYR"] = sdd
    out["source_file"] = f"{filebase}.xpt"
    out["source_folder"] = folder
    frames.append(out)

sleep_all = pd.concat(frames, ignore_index=True)

# ----------------- preview -----------------
# print("---- Sleep table preview ----")
# print(sleep_all.head(8))
# print("\nCounts by SDDSRVYR:")
# print(sleep_all["SDDSRVYR"].value_counts().sort_index())
print("\nSLQ050 value counts (raw):")
print(sleep_all["SLQ050"].value_counts(dropna=False))
print("\nTrouble sleeping rate by cycle (unweighted):")
print(sleep_all.groupby("SDDSRVYR")["trouble_sleeping"].mean())
print("\nMissingness (key cols):")
print(sleep_all[["SLQ050","sleep_hours_weekday","sleep_hours_weekend","sleep_hours_usual"]].isna().mean().round(3))

# ----------------- merge (no save) -----------------
# 1) keep only what you want from sleep_all
slq_min = sleep_all[["SEQN", "SDDSRVYR", "trouble_sleeping", "sleep_hours_usual"]].drop_duplicates()

# 2) make sure key dtypes match on both sides (prevents silent non-matches)
for df_ in (df_my_cov_aligned_short, slq_min):
    df_["SEQN"] = pd.to_numeric(df_["SEQN"], errors="coerce").astype("Int64")
    df_["SDDSRVYR"] = pd.to_numeric(df_["SDDSRVYR"], errors="coerce").astype("Int64")

# 3) merge INTO df_my_cov_aligned_short (overwrites the same variable)
before = len(df_my_cov_aligned_short)
df_my_cov_aligned_short = df_my_cov_aligned_short.merge(
    slq_min, on=["SEQN","SDDSRVYR"], how="left", validate="m:1"
)

# 4) quick checks
print(f"Rows before: {before} | after: {len(df_my_cov_aligned_short)} (should match)")
print("Non-missing coverage (any of the two):",
      ((df_my_cov_aligned_short[['trouble_sleeping','sleep_hours_usual']].notna()).any(axis=1)).mean().round(3))
print(df_my_cov_aligned_short.head(8)[["SEQN","SDDSRVYR","trouble_sleeping","sleep_hours_usual"]])

# (saving is still commented out until you say go)
# from pathlib import Path
# OUT = Path('/Users/dengshuyue/Desktop/SDOH/analysis/output'); OUT.mkdir(parents=True, exist_ok=True)
# df_merged.to_parquet(OUT/'cov_addvX_99_23_with_sleep.parquet', index=False)
# df_merged.to_csv(OUT/'cov_addvX_99_23_with_sleep.csv', index=False)



SLQ050 value counts (raw):
SLQ050
2.0    41683
1.0    13173
NaN     8502
9.0       33
7.0        6
Name: count, dtype: int64

Trouble sleeping rate by cycle (unweighted):
SDDSRVYR
4     0.193148
5     0.219878
6     0.232501
7      0.22359
8     0.244816
9     0.257631
10     0.26332
12        <NA>
66    0.268774
Name: trouble_sleeping, dtype: Float64

Missingness (key cols):
SLQ050                 0.134
sleep_hours_weekday    0.005
sleep_hours_weekend    0.612
sleep_hours_usual      0.005
dtype: float64
Rows before: 128809 | after: 128809 (should match)
Non-missing coverage (any of the two): 0.491
   SEQN  SDDSRVYR  trouble_sleeping  sleep_hours_usual
0     1         1              <NA>                NaN
1     2         1              <NA>                NaN
2     3         1              <NA>                NaN
3     4         1              <NA>                NaN
4     5         1              <NA>                NaN
5     6         1              <NA>                NaN
6     7 

#### sanity check

In [33]:
df_my_cov_aligned_short.columns

Index(['SEQN', 'SDDSRVYR', 'sdmvpsu', 'sdmvstra', 'RIDAGEYR', 'SEX', 'RACE',
       're', 'household_size', 'DMDHHSIZ', 'EDU', 'pir', 'met_hr',
       'SMK_STATUS', 'CIGS_PER_DAY', 'FORMER_SMOKER', 'METSCORE', 'LTPA',
       'DIABE', 'HYPERTEN', 'chol_rx', 'CVD', 'cancer', 'probable_depression',
       'ahei_total', 'unemployment2', 'ins', 'HOQ065', 'marriage', 'SNAP',
       'FS', 'WTINT2YR', 'WTMEC2YR', 'WTSAF2YR', 'WTINT4YR', 'WTMEC4YR',
       'WTPH2YR', 'WTINTPRP', 'WTMECPRP', 'WTSAFPRP', 'wt_int', 'wt_mec',
       'wt_fasting', 'wt_phlebotomy', 'marriage_prev', 'marriage_label',
       'marriage3', 'SNAP_src', 'SNAP_bin', 'SNAP_src_rank', 'SNAP_indiv_only',
       'SNAP_indiv_plus_singleton', 'bmi', 'RIAGENDR', 'drinking', 'alcg2',
       'perE_alco', 'METSCORE_fromPAQ', 'LTPA_fromPAQ', 'chol_rx200',
       'SMK_STATUS_STD', 'CIGS_PER_DAY_CAT', 'bmi_lu', 'bmi_cat_new',
       'trouble_sleeping', 'sleep_hours_usual'],
      dtype='object')

In [34]:
# Per-cycle missingness (only the two requested columns)
df_my_cov_aligned_short["SDDSRVYR"] = pd.to_numeric(df_my_cov_aligned_short["SDDSRVYR"], errors="coerce")

miss = (
    df_my_cov_aligned_short
    .groupby("SDDSRVYR")[["trouble_sleeping", "sleep_hours_usual"]]
    .apply(lambda d: pd.Series({
        "miss_trouble_pct": d["trouble_sleeping"].isna().mean(),
        "miss_hours_pct":   d["sleep_hours_usual"].isna().mean(),
    }))
    .reset_index()
    .sort_values("SDDSRVYR")
)

print(miss.round(3))
# If you prefer 0–100% instead of 0–1, use: (miss[["miss_trouble_pct","miss_hours_pct"]] * 100).round(1)


    SDDSRVYR  miss_trouble_pct  miss_hours_pct
0          1             1.000           1.000
1          2             1.000           1.000
2          3             1.000           1.000
3          4             0.408           0.407
4          5             0.356           0.355
5          6             0.346           0.347
6          7             0.367           0.368
7          8             0.365           0.365
8          9             0.366           0.369
9         10             0.335           0.339
10        12             1.000           0.297
11        66             0.345           0.351


#### (not use) just check sleep time for 2021

In [35]:
# ===== SLQ300 summary for SDDSRVYR = 12 (2021–2023) =====
def _hhmm_to_min_safe(s):
    try:
        s = str(s)
        if ":" not in s:
            return np.nan
        h, m = s.split(":")
        return int(h) * 60 + int(m)
    except Exception:
        return np.nan

def _min_to_hhmm(x):
    h = int(x) // 60
    m = int(x) % 60
    return f"{h:02d}:{m:02d}"

if "SLQ300" not in sleep_all.columns:
    print("SLQ300 not present in the assembled sleep table.")
else:
    slq12 = sleep_all.loc[sleep_all["SDDSRVYR"] == 12].copy()
    raw = slq12["SLQ300"].astype(str)

    # Count special/coded/missing
    refused = (raw == "77777").sum()
    dontknow = (raw == "99999").sum()
    is_time = raw.str.contains(":", na=False)
    n_time = int(is_time.sum())
    n_total = int(len(slq12))
    n_missing_other = n_total - n_time - refused - dontknow

    # Parse HH:MM to minutes for stats
    mins = raw.where(is_time).apply(_hhmm_to_min_safe).dropna()
    if len(mins):
        min_min = int(mins.min())
        max_min = int(mins.max())
        mean_min = int(round(mins.mean()))
        print("\nSLQ300 (Usual sleep time on weekdays), SDDSRVYR=12 (2021–2023)")
        print(f"  total rows:            {n_total}")
        print(f"  valid HH:MM times:     {n_time}")
        print(f"  refused (77777):       {refused}")
        print(f"  don't know (99999):    {dontknow}")
        print(f"  blank/other missing:   {n_missing_other}")
        print(f"  min time:              {_min_to_hhmm(min_min)}")
        print(f"  max time:              {_min_to_hhmm(max_min)}")
        print(f"  mean time:             {_min_to_hhmm(mean_min)}")
    else:
        print("\nSLQ300 summary: no parsable HH:MM values for SDDSRVYR=12.")

# Optional: peek at a few raw SLQ300 values for SDDSRVYR=12
print("\nSample SLQ300 values (SDDSRVYR=12):")
print(sleep_all.loc[sleep_all["SDDSRVYR"] == 12, "SLQ300"].head(15))



SLQ300 (Usual sleep time on weekdays), SDDSRVYR=12 (2021–2023)
  total rows:            8501
  valid HH:MM times:     8346
  refused (77777):       6
  don't know (99999):    82
  blank/other missing:   67
  min time:              00:00
  max time:              23:45
  mean time:             16:34

Sample SLQ300 values (SDDSRVYR=12):
54896    21:30
54897    21:00
54898    00:00
54899    21:30
54900    22:05
54901    00:00
54902    04:00
54903    21:00
54904    23:00
54905    22:00
54906    03:00
54907    22:30
54908    23:30
54909    22:00
54910    00:00
Name: SLQ300, dtype: object


## add 1999-2004 anxiety socre as a proxy 

In [36]:
import pandas as pd, numpy as np, io, requests
from typing import Optional

# ---------------- helpers ----------------
def _fetch_xpt(folder: str, filebase: str) -> pd.DataFrame:
    url = f"https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/{folder}/DataFiles/{filebase}.xpt"
    r = requests.get(url, timeout=60); r.raise_for_status()
    df = pd.read_sas(io.BytesIO(r.content), format="xport", encoding="latin1")
    df.columns = [c.upper() for c in df.columns]
    return df

def _safe_fetch(folder: str, filebase: str) -> Optional[pd.DataFrame]:
    try:
        return _fetch_xpt(folder, filebase)
    except Exception as e:
        print(f"[skip] {folder}/{filebase}.xpt — {type(e).__name__}: {e}")
        return None

def _pick_score_col(cols):
    if "CIDGSCOR" in cols: return "CIDGSCOR"
    # fallback heuristic
    cand = [c for c in cols if c.startswith("CID") and c.endswith("SCOR")]
    return cand[0] if cand else None

def _pick_cidi_weight(cols):
    # choose a CIDI-specific 2-yr weight if present
    cands = [c for c in cols if c.startswith("WT") and c.endswith("2YR") and c not in ("WTMEC2YR","WTINT2YR")]
    return cands[0] if cands else None

def _prep_ciqgad(df: pd.DataFrame, sddsrvyr: int) -> pd.DataFrame:
    out = df.copy()
    # optional age filter (module targets 20–39)
    if "RIDAGEYR" in out.columns:
        out = out[(out["RIDAGEYR"] >= 20) & (out["RIDAGEYR"] <= 39)].copy()

    score_col = _pick_score_col(out.columns)
    if score_col is None:
        # ensure column exists even if not found
        out["CIDGSCOR"] = np.nan
    elif score_col != "CIDGSCOR":
        out.rename(columns={score_col: "CIDGSCOR"}, inplace=True)

    # keep only essentials + weight passthrough
    keep = ["SEQN", "CIDGSCOR"]
    wcol = _pick_cidi_weight(out.columns)
    if wcol:
        keep.append(wcol)
        out.rename(columns={wcol: "WT_CIDI_2YR"}, inplace=True)
        keep[-1] = "WT_CIDI_2YR"

    out = out[keep].copy()
    out["SDDSRVYR"] = sddsrvyr

    # normalize types
    out["SEQN"] = pd.to_numeric(out["SEQN"], errors="coerce").astype("Int64")
    out["CIDGSCOR"] = pd.to_numeric(out["CIDGSCOR"], errors="coerce")
    if "WT_CIDI_2YR" in out.columns:
        out["WT_CIDI_2YR"] = pd.to_numeric(out["WT_CIDI_2YR"], errors="coerce")

    # convenient indicator: 1 if positive diagnosis (per your codebook), 0 if negative, NA otherwise
    out["anxiety_pos"] = pd.Series(
        np.where(out["CIDGSCOR"] == 1, 1,
        np.where(out["CIDGSCOR"] == 5, 0, np.nan))
    ).astype("Int64")

    return out

# ---------------- fetch 1999–2004 ----------------
gad_targets = [
    ("1999", "CIQGAD",   1),  # 1999–2000
    ("2001", "CIQGAD_B", 2),  # 2001–2002 (may 404 if not present—safe)
    ("2003", "CIQGAD_C", 3),  # 2003–2004
]

gad_frames = []
for folder, filebase, sdd in gad_targets:
    df = _safe_fetch(folder, filebase)
    if df is None:
        continue
    out = _prep_ciqgad(df, sdd)
    out["source_file"] = f"{filebase}.xpt"
    out["source_folder"] = folder
    gad_frames.append(out)

if not gad_frames:
    raise RuntimeError("No CIQGAD files fetched.")

gad_all = pd.concat(gad_frames, ignore_index=True)

# ---------------- preview ----------------
print("---- CIDI GAD preview ----")
print(gad_all.head(10))

print("\nCounts by SDDSRVYR:")
print(gad_all["SDDSRVYR"].value_counts().sort_index())

print("\nCIDGSCOR value counts (raw):")
print(gad_all["CIDGSCOR"].value_counts(dropna=False))

print("\nAnxiety positive rate by cycle (unweighted):")
print(gad_all.groupby("SDDSRVYR")["anxiety_pos"].mean())

print("\nMissingness:")
print(gad_all[["CIDGSCOR","anxiety_pos"]].isna().mean().round(3))

# ---------------- merge into df_my_cov_aligned_short (no save) ----------------
# keep only what we want to attach
gad_min = gad_all[["SEQN","SDDSRVYR","CIDGSCOR","anxiety_pos"]].drop_duplicates()

# align key dtypes on both sides
for df_ in (df_my_cov_aligned_short, gad_min):
    df_["SEQN"] = pd.to_numeric(df_["SEQN"], errors="coerce").astype("Int64")
    df_["SDDSRVYR"] = pd.to_numeric(df_["SDDSRVYR"], errors="coerce").astype("Int64")

before = len(df_my_cov_aligned_short)
df_my_cov_aligned_short = df_my_cov_aligned_short.merge(
    gad_min, on=["SEQN","SDDSRVYR"], how="left", validate="m:1"
)

print(f"\nRows before: {before} | after: {len(df_my_cov_aligned_short)} (should match)")
print("Merge coverage (CIDGSCOR non-missing):",
      df_my_cov_aligned_short["CIDGSCOR"].notna().mean().round(3))

print("\nMerged preview:")
print(df_my_cov_aligned_short.head(12)[["SEQN","SDDSRVYR","CIDGSCOR","anxiety_pos"]])


---- CIDI GAD preview ----
   SEQN  CIDGSCOR    WT_CIDI_2YR  SDDSRVYR  anxiety_pos source_file  \
0    12       5.0  186078.314366         1            0  CIQGAD.xpt   
1    20       5.0   32784.751140         1            0  CIQGAD.xpt   
2    34       5.0   50386.248286         1            0  CIQGAD.xpt   
3    66       5.0   54383.039478         1            0  CIQGAD.xpt   
4    69       5.0   78866.089966         1            0  CIQGAD.xpt   
5    81       5.0  176135.722403         1            0  CIQGAD.xpt   
6    97       5.0   86334.258174         1            0  CIQGAD.xpt   
7   107       5.0   37883.834696         1            0  CIQGAD.xpt   
8   120       5.0  186788.235958         1            0  CIQGAD.xpt   
9   143       5.0    4969.041484         1            0  CIQGAD.xpt   

  source_folder  
0          1999  
1          1999  
2          1999  
3          1999  
4          1999  
5          1999  
6          1999  
7          1999  
8          1999  
9          

#### sanity check 

In [37]:
# Simple missingness + anxiety rate by cycle (interviewed sample only)
miss = (
    gad_all.groupby("SDDSRVYR")["CIDGSCOR"]
    .apply(lambda s: s.isna().mean())
    .rename("miss_cidgscor_pct")
    .reset_index()
)

rate = (
    gad_all.dropna(subset=["CIDGSCOR"])
    .groupby("SDDSRVYR")["anxiety_pos"]
    .mean()
    .rename("anxiety_rate")
    .reset_index()
)

out = miss.merge(rate, on="SDDSRVYR", how="left").sort_values("SDDSRVYR")
print(out.round(3))


   SDDSRVYR  miss_cidgscor_pct  anxiety_rate
0         1              0.081         0.027
1         2              0.124         0.023
2         3              0.185         0.017


#### quick rename

In [39]:
# rename anxiety_pos → anxiety_rx (dtype preserved)
df_my_cov_aligned_short = df_my_cov_aligned_short.rename(
    columns={"anxiety_pos": "anxiety_rx"}
)

In [40]:
df_my_cov_aligned_short.columns

Index(['SEQN', 'SDDSRVYR', 'sdmvpsu', 'sdmvstra', 'RIDAGEYR', 'SEX', 'RACE',
       're', 'household_size', 'DMDHHSIZ', 'EDU', 'pir', 'met_hr',
       'SMK_STATUS', 'CIGS_PER_DAY', 'FORMER_SMOKER', 'METSCORE', 'LTPA',
       'DIABE', 'HYPERTEN', 'chol_rx', 'CVD', 'cancer', 'probable_depression',
       'ahei_total', 'unemployment2', 'ins', 'HOQ065', 'marriage', 'SNAP',
       'FS', 'WTINT2YR', 'WTMEC2YR', 'WTSAF2YR', 'WTINT4YR', 'WTMEC4YR',
       'WTPH2YR', 'WTINTPRP', 'WTMECPRP', 'WTSAFPRP', 'wt_int', 'wt_mec',
       'wt_fasting', 'wt_phlebotomy', 'marriage_prev', 'marriage_label',
       'marriage3', 'SNAP_src', 'SNAP_bin', 'SNAP_src_rank', 'SNAP_indiv_only',
       'SNAP_indiv_plus_singleton', 'bmi', 'RIAGENDR', 'drinking', 'alcg2',
       'perE_alco', 'METSCORE_fromPAQ', 'LTPA_fromPAQ', 'chol_rx200',
       'SMK_STATUS_STD', 'CIGS_PER_DAY_CAT', 'bmi_lu', 'bmi_cat_new',
       'trouble_sleeping', 'sleep_hours_usual', 'CIDGSCOR', 'anxiety_rx'],
      dtype='object')

## Save 

In [43]:
from pathlib import Path

out_path = Path("/Users/dengshuyue/Desktop/SDOH/analysis/output/cov_addv7_99_23.parquet")
out_path.parent.mkdir(parents=True, exist_ok=True)

# Save with snappy compression (good default); requires pyarrow or fastparquet installed
df_my_cov_aligned_short.to_parquet(out_path, index=False, compression="snappy")
print(f"Saved: {out_path}")


Saved: /Users/dengshuyue/Desktop/SDOH/analysis/output/cov_addv7_99_23.parquet
